In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from datetime import datetime, timedelta
from tqdm import tqdm

from matplotlib.dates import DateFormatter
import warnings
warnings.filterwarnings("ignore")

from functions import print_shape

#### helper functions

In [2]:
from functions import (
    missingValues
)

In [3]:
def date_range(df, date='Date'):
    date = df.date
    date_desc = date.describe(datetime_is_numeric=True)
    date_min = date_desc.loc['min']
    date_max = date_desc.loc['max']
    print('Data from {:%Y-%m-%d} to {:%Y-%m-%d}'
            .format(date_min, date_max))
    print('Data observations {} '.format(
                    date.nunique()))

# Data


## Stock_list

In [4]:
stock_list = pd.read_csv('data/stock_list.csv', parse_dates=['EffectiveDate', 'TradeDate'])

In [5]:
stock_list.head(2)

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,2021-12-30,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True
1,1305,2021-12-30,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False


In [6]:
stock_list.SecuritiesCode.nunique()

4417

In [7]:
stock_list.sort_values('MarketCapitalization', ascending=False).head(10)

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
137,1557,2021-12-30,SPDR S&P500 ETF Trust,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,54920.0,9.503321e+08,5.219224e+13,False
3065,7203,2021-12-30,TOYOTA MOTOR CORPORATION,First Section (Domestic),Prime Market,3700,Transportation Equipment,6,AUTOMOBILES & TRANSPORTATION EQUIPMENT,1,TOPIX Core30,20211230.0,2105.5,1.631499e+10,3.435121e+13,True
2804,6758,2021-12-30,SONY GROUP CORPORATION,First Section (Domestic),Prime Market,3650,Electric Appliances,9,ELECTRIC APPLIANCES & PRECISION INSTRUMENTS,1,TOPIX Core30,20211230.0,14475.0,1.261082e+09,1.825416e+13,True
2867,6861,2021-12-30,KEYENCE CORPORATION,First Section (Domestic),Prime Market,3650,Electric Appliances,9,ELECTRIC APPLIANCES & PRECISION INSTRUMENTS,1,TOPIX Core30,20211230.0,72280.0,2.432077e+08,1.757905e+13,True
2,1306,2021-12-30,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False
2375,6098,2021-12-30,"Recruit Holdings Co.,Ltd.",First Section (Domestic),Prime Market,9050,Services,10,"IT & SERVICES, OTHERS",1,TOPIX Core30,20211230.0,6972.0,1.695960e+09,1.182423e+13,True
4158,9432,2021-12-30,NIPPON TELEGRAPH AND TELEPHONE CORPORATION,First Section (Domestic),Prime Market,5250,Information & Communication,10,"IT & SERVICES, OTHERS",1,TOPIX Core30,20211230.0,3150.0,3.622013e+09,1.140934e+13,True
3577,8035,2021-12-30,Tokyo Electron Limited,First Section (Domestic),Prime Market,3650,Electric Appliances,9,ELECTRIC APPLIANCES & PRECISION INSTRUMENTS,1,TOPIX Core30,20211230.0,66280.0,1.572109e+08,1.041994e+13,True
4404,9984,2021-12-30,SoftBank Group Corp.,First Section (Domestic),Prime Market,5250,Information & Communication,10,"IT & SERVICES, OTHERS",1,TOPIX Core30,20211230.0,5434.0,1.722954e+09,9.362531e+12,True
1508,4063,2021-12-30,"Shin-Etsu Chemical Co.,Ltd.",First Section (Domestic),Prime Market,3200,Chemicals,4,RAW MATERIALS & CHEMICALS,1,TOPIX Core30,20211230.0,19920.0,4.166628e+08,8.299923e+12,True


In [8]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SecuritiesCode          4417 non-null   int64         
 1   EffectiveDate           4417 non-null   datetime64[ns]
 2   Name                    4417 non-null   object        
 3   Section/Products        4417 non-null   object        
 4   NewMarketSegment        3772 non-null   object        
 5   33SectorCode            4417 non-null   object        
 6   33SectorName            4417 non-null   object        
 7   17SectorCode            4417 non-null   object        
 8   17SectorName            4417 non-null   object        
 9   NewIndexSeriesSizeCode  4417 non-null   object        
 10  NewIndexSeriesSize      4417 non-null   object        
 11  TradeDate               4121 non-null   object        
 12  Close                   4121 non-null   float64 

In [9]:
stock_list.TradeDate = stock_list.TradeDate.astype(float)
#stock_list.TradeDate = stock_list.TradeDate.astype(int)
#stock_list['trade_date'] = pd.to_datetime(stock_list.TradeDate)
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SecuritiesCode          4417 non-null   int64         
 1   EffectiveDate           4417 non-null   datetime64[ns]
 2   Name                    4417 non-null   object        
 3   Section/Products        4417 non-null   object        
 4   NewMarketSegment        3772 non-null   object        
 5   33SectorCode            4417 non-null   object        
 6   33SectorName            4417 non-null   object        
 7   17SectorCode            4417 non-null   object        
 8   17SectorName            4417 non-null   object        
 9   NewIndexSeriesSizeCode  4417 non-null   object        
 10  NewIndexSeriesSize      4417 non-null   object        
 11  TradeDate               4121 non-null   float64       
 12  Close                   4121 non-null   float64 

In [10]:
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
sec_codes = prices.SecuritiesCode.unique()
del prices

In [11]:
date = stock_list.EffectiveDate
date_desc = date.describe(datetime_is_numeric=True)
date_min = date_desc.loc['min']
date_max = date_desc.loc['max']
print('Data from {:%Y-%m-%d} to {:%Y-%m-%d}'
            .format(date_min, date_max))
print('Data observations {} '.format(
                    date.nunique()))

Data from 2000-01-01 to 2022-02-25
Data observations 10 


In [12]:
del stock_list

## Financials

In [13]:
financial = pd.read_csv('data/train_files/financials.csv', parse_dates=['Date'])

In [14]:
financial.head(2)

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,2.016121e+13,20170104_2753,2017-01-04,2753.0,2017-01-04,07:30:00,1.483483e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,319.76,NaN,False,True,False,False,False,6848800.0,－,6848800.0
1,2.017010e+13,20170104_3353,2017-01-04,3353.0,2017-01-04,15:00:00,1.483510e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,485.36,NaN,False,True,False,False,False,2035000.0,118917,1916083.0


In [15]:
# 2761
financial.query('SecuritiesCode == 2761').tail()

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
69236,2.020093e+13,20201002_2761,2020-10-02,2761.0,2020-10-02,11:00:00,1.601604e+09,1QFinancialStatements_Consolidated_JP,2020-08-20,1Q,...,211.22,NaN,False,False,False,False,False,11405400.0,3451825,7953575
74109,2.020113e+13,20201228_2761,2020-12-28,2761.0,2020-12-28,11:00:00,1.609121e+09,2QFinancialStatements_Consolidated_JP,2020-11-20,2Q,...,211.22,NaN,False,False,False,False,False,11405400.0,3453425,7953018
79003,2.021031e+13,20210331_2761,2021-03-31,2761.0,2021-03-31,11:00:00,1.617156e+09,3QFinancialStatements_Consolidated_JP,2021-02-20,3Q,...,211.22,NaN,False,False,False,False,False,11405400.0,3453425,7952673
84011,2.021062e+13,20210702_2761,2021-07-02,2761.0,2021-07-02,11:00:00,1.625191e+09,FYFinancialStatements_Consolidated_JP,2021-05-20,FY,...,193.72,NaN,False,False,False,False,False,11405400.0,3453425,7952501.0
88535,2.021092e+13,20211001_2761,2021-10-01,2761.0,2021-10-01,11:00:00,1.633054e+09,1QFinancialStatements_Consolidated_JP,2021-08-20,1Q,...,193.72,NaN,False,True,False,False,False,11405400.0,3457125,7950035.0


In [16]:
financial.columns

Index(['DisclosureNumber', 'DateCode', 'Date', 'SecuritiesCode',
       'DisclosedDate', 'DisclosedTime', 'DisclosedUnixTime', 'TypeOfDocument',
       'CurrentPeriodEndDate', 'TypeOfCurrentPeriod',
       'CurrentFiscalYearStartDate', 'CurrentFiscalYearEndDate', 'NetSales',
       'OperatingProfit', 'OrdinaryProfit', 'Profit', 'EarningsPerShare',
       'TotalAssets', 'Equity', 'EquityToAssetRatio', 'BookValuePerShare',
       'ResultDividendPerShare1stQuarter', 'ResultDividendPerShare2ndQuarter',
       'ResultDividendPerShare3rdQuarter',
       'ResultDividendPerShareFiscalYearEnd', 'ResultDividendPerShareAnnual',
       'ForecastDividendPerShare1stQuarter',
       'ForecastDividendPerShare2ndQuarter',
       'ForecastDividendPerShare3rdQuarter',
       'ForecastDividendPerShareFiscalYearEnd',
       'ForecastDividendPerShareAnnual', 'ForecastNetSales',
       'ForecastOperatingProfit', 'ForecastOrdinaryProfit', 'ForecastProfit',
       'ForecastEarningsPerShare',
       'ApplyingOf

### starting date

In [17]:
# drop 
financial = financial.query('TypeOfDocument != ["ForecastRevision", "ForecastRevision_REIT"]')

In [18]:
financial.head()

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,2.016121e+13,20170104_2753,2017-01-04,2753.0,2017-01-04,07:30:00,1.483483e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,319.76,NaN,False,True,False,False,False,6848800.0,－,6848800.0
1,2.017010e+13,20170104_3353,2017-01-04,3353.0,2017-01-04,15:00:00,1.483510e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,485.36,NaN,False,True,False,False,False,2035000.0,118917,1916083.0
3,2.017010e+13,20170105_2659,2017-01-05,2659.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,285.05,NaN,False,True,False,False,False,31981654.0,18257,31963405.0
7,2.017011e+13,20170105_3344,2017-01-05,3344.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,47.7,NaN,False,True,False,False,False,5579184.0,1372,5578257.0
9,2.017010e+13,20170105_7453,2017-01-05,7453.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,913.28,NaN,False,True,True,False,False,28078000.0,1702372,26527712.0


In [19]:
liste =[ 'Date', 'SecuritiesCode', 'Profit', 'NetSales', 'OperatingProfit', 'BookValuePerShare',
        'ForecastDividendPerShareFiscalYearEnd',
        'ForecastDividendPerShareAnnual', 'ForecastNetSales',
        'ForecastOperatingProfit', 'ForecastOrdinaryProfit', 'ForecastProfit',
        'ForecastEarningsPerShare']

financial = financial[liste]

In [20]:
financial.head()

,Date,SecuritiesCode,Profit,NetSales,OperatingProfit,BookValuePerShare,ForecastDividendPerShareFiscalYearEnd,ForecastDividendPerShareAnnual,ForecastNetSales,ForecastOperatingProfit,ForecastOrdinaryProfit,ForecastProfit,ForecastEarningsPerShare
0,2017-01-04,2753.0,1494000000,22761000000,2147000000,2671.42,50.0,100.0,31800000000,3255000000,3300000000,2190000000,319.76
1,2017-01-04,3353.0,629000000,22128000000,820000000,NaN,36.0,72.0,30200000000,1350000000,1300000000,930000000,485.36
3,2017-01-05,2659.0,7171000000,134781000000,11248000000,3073.12,42.0,42.0,177683000000,14168000000,14473000000,9111000000,285.05
7,2017-01-05,3344.0,-669000000,54452000000,-541000000,NaN,25.0,25.0,75500000000,500000000,650000000,250000000,47.7
9,2017-01-05,7453.0,19996000000,247027000000,29651000000,5350.88,137.0,274.0,336500000000,38000000000,38000000000,24300000000,913.28


In [21]:
#financial.SecuritiesCode.nunique()
# Selecting only matching Codes from financial data
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
sec_codes = [x for x in prices.SecuritiesCode.unique() if x in financial.SecuritiesCode.unique()]


In [22]:
print_shape(financial)

 Shape:
 ----------------------------------------
 Observations:   75.6K
 Features:       13
 Feature Date:    datetime64[ns]
----------------------------------------
Index(['Date', 'SecuritiesCode', 'Profit', 'NetSales', 'OperatingProfit',
       'BookValuePerShare', 'ForecastDividendPerShareFiscalYearEnd',
       'ForecastDividendPerShareAnnual', 'ForecastNetSales',
       'ForecastOperatingProfit', 'ForecastOrdinaryProfit', 'ForecastProfit',
       'ForecastEarningsPerShare'],
      dtype='object')
----------------------------------------
Date missing Values: 0 (0.0%)
SecuritiesCode missing Values: 2 (0.0%)
Profit missing Values: 261 (0.3%)
NetSales missing Values: 193 (0.3%)
OperatingProfit missing Values: 2195 (2.9%)
BookValuePerShare missing Values: 39868 (52.7%)
ForecastDividendPerShareFiscalYearEnd missing Values: 1584 (2.1%)
ForecastDividendPerShareAnnual missing Values: 1583 (2.1%)
ForecastNetSales missing Values: 7045 (9.3%)
ForecastOperatingProfit missing Values: 8362 (11.1

In [23]:
#TODO es funktioniert nicht!

In [25]:
droped_days_dict = {}


def droped_days (df, except_col = 'ExpectedDividends'):

    col_to_drop = [x for x in df.columns if x != except_col]
    df_drop = df.dropna(axis=0, subset = col_to_drop)
    
    first_day = df.Date.iloc[0].date()
    #display(df_drop.head())
    first_day_drop = df_drop.Date.iloc[0].date()
    days_drop =  first_day_drop - first_day 

    return first_day, first_day_drop, days_drop.days


for code in tqdm(sec_codes[:100]):
    #print(code)
    df_f = financial.query('SecuritiesCode == @code').sort_values('Date')
    f_first_day, f_first_day_drop, f_days_drop = droped_days(df_f)

    df_p = prices.query('SecuritiesCode == @code').sort_values('Date')    
    p_first_day, p_first_day_drop, p_days_drop = droped_days(df_p)
    #display(df_p.head())
    financial_preis_days_drop = p_first_day_drop - f_first_day_drop

    droped_days_dict[code] = (f_first_day, f_first_day_drop, f_days_drop, p_first_day, p_first_day_drop, p_days_drop, financial_preis_days_drop )


#droped_df = pd.DataFrame(droped_days).T
#droped_df.columns = ['F_first_day', 'F_firest_day_drop', 'F_days', 'P_first_day', 'P_firest_day_drop', 'P_days']

100%|██████████| 100/100 [00:00<00:00, 115.27it/s]


In [26]:
from functions import df_security_code
d2120 = df_security_code(prices, code=2120)
d2120.head()
df_drop = d2120.dropna(axis=0)
df_drop.Date.iloc[0].date()

datetime.date(2017, 3, 27)

In [27]:
for i in tqdm(sec_codes):
    d212_ = df_security_code(prices, i)
    d212_.head()
    df_drop = d212_.dropna(axis=0)
    #print(i, '  ', df_drop.Date.iloc[0].date())


100%|██████████| 2000/2000 [00:12<00:00, 156.62it/s]


In [28]:
droped_df = pd.DataFrame(droped_days_dict).T
droped_df.columns = ['F_first_day', 'F_firest_day_drop', 'F_days', 'P_first_day', 'P_firest_day_drop', 'P_days', 'total']
droped_df

,F_first_day,F_firest_day_drop,F_days,P_first_day,P_firest_day_drop,P_days,total
1301,2017-02-10,2017-05-11,90,2017-01-04,2017-03-27,82,-45 days
1332,2017-01-20,2017-01-20,0,2017-01-04,2017-03-27,82,66 days
1333,2017-02-06,2017-05-08,91,2017-01-04,2017-03-27,82,-42 days
1376,2017-01-06,2017-01-06,0,2017-01-04,2017-05-25,141,139 days
1377,2017-01-12,2017-07-13,182,2017-01-04,2017-05-25,141,-49 days
...,...,...,...,...,...,...,...
1979,2017-02-10,2017-05-15,94,2017-01-04,2017-03-27,82,-49 days
1980,2017-02-08,2017-02-08,0,2017-01-04,2017-03-27,82,47 days
1981,2017-02-08,2017-05-12,93,2017-01-04,2017-03-27,82,-46 days
1982,2017-02-07,2017-02-07,0,2017-01-04,2017-03-27,82,48 days


In [ ]:
df_1332 = prices.query('SecuritiesCode == @code').sort_values('Date')
df_1332.head()

In [ ]:
for code in sec_codes[99:101]:
    print(code)
    df_f = financial.query('SecuritiesCode == @code').sort_values('Date')
    df_p = prices.query('SecuritiesCode == @code').sort_values('Date')
    display(df_f.head())
    display(df_p.head())
    

In [ ]:
prices_droped_day = {}

for code in sec_codes[:2]:
    df = prices.query('SecuritiesCode == @code').sort_values('Date')
    df_drop = df.dropna(axis=0)
    print(code)
    first_day = df.Date.iloc[0].date()

    first_day_drop = df_drop.Date.iloc[0].date()
    days_drop =  first_day_drop -first_day 
    prices_droped_day[code] = [first_day, first_day_drop, days_drop.days]
    #print(f'first date before drop: {first_day:%Y-%m-%d}')
    #print(f'first date after drop:  {first_day_drop:%Y-%m-%d}')
    #print(f'droped days:  {days_drop.days},                 TAKE care NOT working days')

In [ ]:
pd.DataFrame(prices_droped_day).T

In [ ]:
del financial

## Options

In [ ]:
options = pd.read_csv('data/train_files/options.csv', parse_dates=[1])

In [ ]:
options.head(2)

In [ ]:
options.columns

- Date

In [ ]:
date_range(options)


- codes : more than securites code

In [ ]:
options.OptionsCode.nunique()

In [ ]:
options.OptionsCode.describe()

In [ ]:
del options

## Submission

In [ ]:
submission = pd.read_csv('data/example_test_files/sample_submission.csv', date_parser=[0])
submission.shape

In [ ]:
submission.nunique()

In [ ]:
submission.head()

Target: calculate the rank for each security code for the next 56 working days

In [ ]:
del submission

## Stock prices

In [ ]:
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
prices.shape

In [ ]:
prices.head(2)

### - date

In [ ]:
date = prices.Date
date_desc = date.describe(datetime_is_numeric=True)
date_min = date_desc.loc['min']
date_max = date_desc.loc['max']
print('Data from {:%Y-%m-%d} to {:%Y-%m-%d}'
        .format(date_min, date_max))
print('Data observations {} '.format(
                 date.nunique()))

from https://www.timeanddate.com/date/workdays.html?d1=4&m1=1&y1=2017&d2=3&m2=12&y2=2021&ti=on&

1211 days 

1795 calendar days – 584 days skipped:
Excluded 256 Saturdays
Excluded 256 Sundays
Excluded 72 holidays

excluded 2010-10-1 because system failer in jpx

remain 1210 days  and 8 left
- Which one are missing and why?

1202 == 1794 . we miss data, where?
- find missing data

- generate working days

In [ ]:
# test handel with timeseries in dataframe
t = np.arange(date_min, date_max, timedelta(days=1)).astype(datetime)

#t_work = [day for day in t if asia.Japan().is_working_day(day)]
date_df = pd.DataFrame(date)
#time_df = pd.to_datetime(t_work)
type(pd.Timestamp(date.unique()[0]))

- observations

In [ ]:
prices.info()

In [ ]:
prices.head(2)

In [ ]:
prices.tail(2)

- amount of trades each day

In [ ]:
count = pd.DataFrame(prices.Date.value_counts().reset_index())
count.head()
plt.figure(figsize = (20,5))
sns.lineplot(data = count, x= 'index', y ='Date')


are stocks added during the time? 
compare the stocks in 1 feb 2017 with the stocks in 1 Feb 2121 

In [ ]:
# stock for each days
feb2017 = prices[prices['Date']=='2017-2-1']
feb2021 = prices[prices['Date']=='2021-2-1']

print('SecuritiesCode for Feb 2017 : {} \nSecuritiesCode for Feb 2021 : {}'. format(
                                    feb2017.SecuritiesCode.nunique(), feb2021.SecuritiesCode.nunique()
))

In [ ]:
# new Securites Code in 2021
codes_2017 = feb2017.SecuritiesCode.to_list()
new_codes = feb2021[~feb2021['SecuritiesCode'].isin(codes_2017)]
old_codes = feb2021[~feb2021['SecuritiesCode'].isin(new_codes.SecuritiesCode.to_list())]
print(new_codes.SecuritiesCode.count())
print(old_codes.SecuritiesCode.count())

In [ ]:
def plot_stock(df, Code, feature='Target'):
    df = df.query('SecuritiesCode==@Code')
    #plt.figure(figsize=(20,5))
    sns.lineplot(data=df, y=feature, x='Date', label=code)

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
for code in [1375, 1301]:
    plot_stock(prices, code)

    ax.xaxis.set_major_formatter(DateFormatter("%m-%Y"))

there are 135 Securities codes not listed in 2017. 
may be they were launch later. 

### nan values

In [ ]:
a = prices[(prices['Date']=='2021-12-03')]
a.count()

In [ ]:
a.query('Open != Open')

a day before

In [ ]:
b = prices[(prices['Date']=='2021-12-02')]
b.query('Open != Open')

and before

In [ ]:
c = prices[(prices['Date']=='2021-12-01')]
c.query('Open != Open')

In [ ]:
d = prices[(prices['Date']=='2021-12-01')]
c.query('Open != Open')

In [ ]:

plt.figure(figsize=(25,10))
sns.lineplot(data = prices.query('SecuritiesCode == 2761 and Date > "2021-11-01"' ), x='Date', y= 'Open')

#sns.lineplot(data = prices.query('SecuritiesCode == 9994 and Date > "2021-11-01"' ), x='Date', y= 'Open')

In [ ]:
plt.figure(figsize=(16,10))
sns.lineplot(data = prices.query('SecuritiesCode == 3540'), x='Date', y= 'Open')
sns.lineplot(data = prices.query('SecuritiesCode == 9994'), x='Date', y= 'Open')


In [ ]:
prices.isnull().sum().sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.heatmap(prices.isna(), cbar=False, cmap="plasma",yticklabels=False)
plt.show()

In [ ]:
prices_nan = prices.query('Open != Open')
prices_nan.shape

In [ ]:
prices_nan.Date.value_counts()

In [ ]:
prices_nan[(prices_nan['Date']!='2020-10-01')].Date.value_counts().sort_values(ascending=True)

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(prices_nan.isna(), cbar=False, cmap="plasma",yticklabels=False)
plt.show()

- forward fill for 1-10-2020
- / consecutive missing values of trades can be removed. the stock was taken out
- forward fill for missing values

In [ ]:
prices_wo_nan = prices.query('Date != "2020-10-01" and Open == Open')

In [ ]:
prices_wo_nan.isnull().sum().sort_values(ascending=False)

### SupervisionFlag
Flag of Securities Under Supervision & Securities to Be Delisted
https://www.jpx.co.jp/english/listing/market-alerts/supervision/00-archives/index.html )


If a stock is designated as Securities Under Supervision or Securities to Be Delisted during the private period, it will be excluded from investment after the date of designation

In [ ]:
prices.head()

In [ ]:
prices.SupervisionFlag.value_counts()

In [ ]:
from functions import plot_stock
w_flag = prices[(prices['SupervisionFlag']==True)]
w_flag.shape


In [ ]:
w_flag.head()

In [ ]:
print(f'Securities codes with flag: {w_flag.SecuritiesCode.nunique()} ')
print(f'Securities codes: {w_flag.SecuritiesCode.unique()} : ')

In [ ]:
codes = w_flag.SecuritiesCode.unique()

for code in codes[:3]:
    plot_stock(prices, code)
    plot_stock(w_flag, code)
    plt.show()

In [ ]:
date = w_flag.Date
date_desc = date.describe(datetime_is_numeric=True)
date_min = date_desc.loc['min']
date_max = date_desc.loc['max']
print('Data from {:%Y-%m-%d} to {:%Y-%m-%d}'
        .format(date_min, date_max))
print('Data observations {} '.format(
                 date.nunique()))

the flags are set for the whole period of time

- how long has a stock a flag?

In [ ]:
flag_day = {}
for code in codes: 
    current = w_flag.query('SecuritiesCode == @ code')
    date = current.Date
    date_desc = date.describe(datetime_is_numeric=True)
    date_min = date_desc.loc['min']
    date_max = date_desc.loc['max']
    flag_day[code] = [date_min, date_max, date.nunique()]


In [ ]:
flag_stats = pd.DataFrame(flag_day).T
flag_stats.columns = ['start_flag', 'end_flag', 'days_w_flag']
flag_stats

encode flag: 

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(prices["SupervisionFlag"])
prices["SupervisionFlag"]=enc.transform(prices["SupervisionFlag"])

In [ ]:
prices.SupervisionFlag.value_counts()

### Volume

In [ ]:
sns.histplot(data=prices, x = 'Volume')

In [ ]:
missingValues(prices.Volume)

In [ ]:
prices['Vol_Close'] = prices['Volume'] * prices['Close']

In [ ]:
volumen_stock = prices.groupby('SecuritiesCode').sum()

In [ ]:
top10 = volumen_stock.sort_values('Vol_Close', ascending=False).reset_index().head(10)

In [ ]:
top10_codes = top10.SecuritiesCode

In [ ]:
stock_list = pd.read_csv('data/stock_list.csv', parse_dates=['EffectiveDate', 'TradeDate'])
stock_list_top10 = stock_list[stock_list['SecuritiesCode'].isin(top10_codes)]
#del stock_list
stock_list_top10

In [ ]:
stock_list_top10 = stock_list[stock_list['SecuritiesCode'].isin(top10_codes)]
stock_list_top10

In [ ]:
stock_list_top10

### Target
Change ratio of adjusted closing price between t+2 and t+1 where t+0 is TradeDate <br>
r(k,t) = C(k, t+2) - C(k, t+1) / C(k, t+1)

- adjusted stock price is used

In [ ]:
sns.histplot(prices.Target, bins=800, )
plt.xlim([-0.4, 0.4])

target for a stock

In [ ]:
def plot_target(df, Code):
    single_stock = df.query('SecuritiesCode==@Code')
    plt.figure(figsize=(20,5))
    sns.lineplot(data=single_stock, y='Close', x='Date')
    ax2 = plt.twinx()
    sns.lineplot(data=single_stock, y='Target', x='Date', color='red')  

In [ ]:
for i in [8876, 6630, 2761, 2753]:
    plot_target(prices, i)

#### - Rank
for each day

In [ ]:
prices.Date.nunique()

In [ ]:
prices_rank = pd.DataFrame(columns = prices.columns)

In [ ]:
# only one day
def calculate_rank(df, day):
    df = prices[(prices['Date']== day)]
    df.loc[:, 'Rank'] = df['Target'].rank(ascending=False, method='first') - 1
    
    return df



df = calculate_rank(prices, prices.Date.unique()[0])
one_day_rank = pd.concat([prices_rank, df], axis=0)



In [ ]:
def daily_spread_return(df, day):
    # calculate rank 
    df = prices[(prices['Date']== day)]
    df.loc[:, 'Rank'] = df['Target'].rank(ascending=False, method='first') - 1

    # calculate weights and weighted_target
    weights = np.linspace(start = 2, stop=1, num=200)

    # sum_up
    top200 = df.sort_values('Rank').iloc[:200,:]
    top200['Weights'] = weights
    top200['weigh_Target'] = top200['Target'] * top200['Weights']
    sum_up = top200['weigh_Target'].sum()/top200['Weights'].mean()

    # sum_down
    bottom200 = one_day_rank.sort_values('Rank').iloc[-200:,:].sort_values('Rank', ascending=False)
    bottom200['Weights'] = np.linspace(start = 2, stop=1, num=200)
    bottom200['weigh_Target'] = bottom200['Target'] * bottom200['Weights']
    sum_down = bottom200['weigh_Target'].sum()/bottom200['Weights'].mean()

    # return daily_spread return
    return  df.SecuritiesCode.nunique(), sum_up - sum_down

In [ ]:
stocks_per_day = {}

def calculate_rank(df, day):
    df = prices[(prices['Date']== day)]
    df.loc[:, 'Rank'] = df['Target'].rank(ascending=False, method='first') - 1
    
    return df

prices_rank = pd.DataFrame(columns = prices.columns)

for day in tqdm(prices.Date.unique()):
    stocks, dsr = daily_spread_return(prices, day)
    stocks_per_day[day] = [stocks, dsr]

In [ ]:
stocks_day = pd.DataFrame(stocks_per_day).T
stocks_day.columns = ['SecuritiesCode_s', 'daily_spread_return']
stocks_day

In [ ]:
stocks_day.nunique()

- best and worst ranks are weighted with 1-2  [link](https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition)

## Trades

In [ ]:
trades = pd.read_csv('data/train_files/trades.csv')
trades.shape
trades.head(2)

In [ ]:
trades.shape


only  nan values?

In [ ]:
del trades

# Features engineering

## 1. fill nan values 

## 2. Adjusted prices
to calculate theoretical price/volume when split/reverse-split happens (NOT including dividend/allotment of shares/)

In [ ]:
prices.AdjustmentFactor.unique()

- amount of stocks for each adjustment factor

In [ ]:

for i in sorted(prices.AdjustmentFactor.unique()):
    print( 'adjustment factor = {}   \t:  {} trades'.format(
        round(i,2), prices[(prices['AdjustmentFactor']==i)].Date.count()
    ))
   

In [ ]:
def adjust_price(df):
    # cumulative adjustment factor considering the day shift
    df.sort_values('Date')
    df.loc[:,'CAF'] = df['AdjustmentFactor'].cumprod().shift(1)
    # fill nan values
    df.CAF.fillna(1, inplace=True)
    # prices to be adjusted
    prices =[ 'Open', 'High', 'Low', 'Close']

    for x in prices:
        df.loc[:,'ad_' + str(x)]  = df[x] / df['CAF']
    
    # adjust volume
    df['ad_Volume'] = df['Volume'] * df['CAF']
    df.drop('CAF', axis=1, inplace=True)

     # adjust target
    df.sort_values('Date')
    df['ad_Close_1'] = df['ad_Close'].shift(-1)
    df['ad_Close_2'] = df['ad_Close'].shift(-2)
    df['ad_Target'] = (df['ad_Close_2']-df['ad_Close_1'])/df['ad_Close_1']
    df.drop(['ad_Close_1', 'ad_Close_2'], axis=1, inplace=True)

    return df

In [ ]:
# create an empty new DataFrame
adjusted_data = pd.DataFrame(columns = prices.columns)

for i in tqdm([8876, 6630, 7453, 7638]): #tqdm(prices.SecuritiesCode.unique()):
    df = prices.query('SecuritiesCode ==@i')
    adjusted_df = adjust_price(df)
    adjusted_data = pd.concat([adjusted_data, adjusted_df ], axis=0)

In [ ]:
def plot_stock(df, Code):
    single_stock = df.query('SecuritiesCode==@Code')
    plt.figure(figsize=(20,5))
    sns.lineplot(data=single_stock, y='Close', x='Date')
    ax2 = plt.twinx()
    sns.lineplot(data=single_stock, y='ad_Close', x='Date', color='red')


In [ ]:
for i in [8876, 6630, 7453, 7638]:
    plot_stock(adjusted_data, i)

data/train_files/stock_prices_ad.csvthe adjusted value was calculated with process_adjusted_price.py and save in : 

In [ ]:
adjusted_data = pd.read_csv('data/train_files/stock_prices_ad.csv', parse_dates=[2], index_col=0)
adjusted_data.head(2)

In [ ]:
def plot_stock(df, Code):
    df = df.query('SecuritiesCode==@Code')
    plt.figure(figsize=(20,5))
    sns.lineplot(data=df, y='Target', x='Date')
    sns.lineplot(data=df, y='ad_Target', x='Date', color = 'red')
    ax2 = plt.twinx()
    sns.lineplot(data=df, y='Close', x='Date')



## 5. Close, Open, High and Low price lag 1


In [ ]:
df.columns

In [ ]:


features = ['ad_Close', 'ad_Open', 'ad_High' , 'ad_Low', 'ad_Volume']
new_features = [x + "_lag1" for x in features]

feat_data = pd.DataFrame(columns = adjusted_data.columns)

def feature_lag(df, features, lag=1):
    for feat in features:
        name = feat + "_lag" +str(lag)
        df[name] = df[str(feat)].shift(lag)
        #new_features.append(name)
    return df


df = adjusted_data.query('SecuritiesCode ==@i')
df = feature_lag(df, features)



In [ ]:
df[new_features + features].head()

## 3. RSI

In [ ]:
def rsi (df_serie,periods = 14, ema = True): 
    """Relative Strength Index
        RSI = 100–(100/1 + RS) ; RS = Av Gain / Av loss
    Args:
        df_serie (_type_): series of ad_Close
        periods (int, optional): 5,7,9,14,21,30 Defaults to 14.
        ema (bool, optional): _description_. Defaults to True.

    Returns:
        pd.Series: _description_
    """
        
    close_delta = df_serie.diff() # .dropna()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)

    if ema == True: # exponential moving average
        ma_up = up.ewm(com= periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    else: # using moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.ewm(window = periods, adjust=False).mean()
        
    rs = ma_up / ma_down
    rsi = 100 - (100 / (1+rs))

    return rsi

In [ ]:
from functions import df_security_code

df = df_security_code(adjusted_data,8194 )
df['RSI'] = rsi(df['ad_Close'])

In [ ]:
from functions import plot_stock

plt.figure(figsize=(20,5))
plot_stock(df, 8194 ,'RSI')
ax2 = plt.twinx()
plot_stock(df, 8194 ,'ad_Close', color='red')

## 4. Return

In [ ]:
df['Return'] = df['ad_Close'].pct_change()
# adjusted_data['Return_m'] = adjusted_data['ad_Close'].resample('M').ffill().pct_change()


In [ ]:
plot_stock(df, 8194, 'Return')

In [ ]:
# cumulative
df['cum_Return'] = (df['Return'] + 1).cumprod()
plot_stock(df, 8194, 'cum_Return')

## 5. Simple moving average





In [ ]:
# ussualy period 10 or 5
# setting for all prices



features = ['ad_Close', 'ad_Open', 'ad_High' , 'ad_Low', 'ad_Volume']
new_features = [x + "_sma10" for x in features]

feat_data = pd.DataFrame(columns = adjusted_data.columns)

def SMA(df, features, period=10):
    for feat in features:
        name = feat + "_sma" + str(period)
        df[name] = df[feat].rolling(window=period).mean()  # only for close or for all values?
        #new_features.append(name)
    return df


df = adjusted_data.query('SecuritiesCode == 8194')
df = SMA(df, features)

In [ ]:
df[features +  new_features].head(20)

## 6. MACD : The Moving Average Convergence Divergence 



In [ ]:

def macd(df):
    ema26 = df['ad_Close'].ewm(span=26, adjust=False, min_periods=26).mean()
    ema12 = df['ad_Close'].ewm(span=12, adjust=False, min_periods=12).mean()

    macd = ema12 - ema26

    # Get the 9-Day EMA of the MACD for the Trigger line

    macd_ema9 = macd.ewm(span=9, adjust=False, min_periods=9).mean()

    # calculate the difference
    macd_diff = macd - macd_ema9

    df['macd'] = df.index.map(macd)
    df['macd_h'] = df.index.map(macd_diff)
    df['macd_s'] = df.index.map(macd_ema9)

    return df

In [ ]:
df = adjusted_data.query('SecuritiesCode == 8194')
df = macd(df)

In [ ]:
df.tail()

In [ ]:

plot_stock(df, 8194, 'ad_Close', color='red')
plot_stock(df, 8194, 'macd')

## 7. Volatility

a value for a period of time. 
ffill


## 8. SPI ? SPY current daily return 

# Preprocessing

## 1. Normalization

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
for i in range(10):
    if i %2:logging.debug(f'{i}This will get logged')



In [ ]:
# TODO include marktcapitalisation from stock list